In [4]:
# a basic neural network without the help from ML libraries

import math
from dataclasses import dataclass
from typing import List, Optional

In [5]:
def sigmoid(x: float) -> float:
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(z: float) -> float:
    return z * (1 - z)

In [6]:
# Neuron Class
# weights, biases, delta (backpropagation variable), output

@dataclass
class Neuron:
    weights: List[float]
    bias: float
    delta: Optional[float] = 0.0
    output: Optional[float] = 0.0

    def _set_output(self, output: float) -> None:
        self.output = output

    # probably not needed as after you set the output, you can call it by self.output
    def get_output(self) -> float:
        return self.output
    
    def set_delta(self, error: float) -> None:
        self.delta = error * sigmoid_derivative(self.output)

    def weighted_sum(self, inputs: List[float]) -> float:
        # since this is a simple NN, I'll just be multiplying the weights and the inputs
        # normally, W should be dot product'ed with the inputs
        ws = self.bias
        for i in range(len(self.weights)):
            ws += self.weights[i] * inputs[i]
        return ws

    # calculate the output of the neuron by feeding the value to a sigmoid func
    def activate(self, inputs: List[float]) -> float:
        output = sigmoid(self.weighted_sum(inputs))
        self._set_output(output)
        return output

In [7]:
# building up the layers

@dataclass
class Layer:
    neurons: List[Neuron]

    @property
    def all_outputs(self) -> List[float]:
        return [neuron.output for neuron in self.neurons]

    def activate_neurons(self, inputs: List[float]) -> List[float]:
        return [neuron.activate(inputs) for neuron in self.neurons]
    
    def total_delta(self, previous_layer_neuron_idx: int) -> float:
        return sum(
            neuron.weights[previous_layer_neuron_idx] * neuron.delta
            for neuron in self.neurons
        )

In [9]:
@dataclass
class Network:
    hidden_layers: List[Layer]
    output_layer: Layer
    learning_rate:  float

    @property
    def layers(self) -> List[Layer]:
        return self.hidden_layers + [self.output_layer]
    
    def feed_forward(self, inputs: List[float]) -> List[float]:
        for layer in self.hidden_layers:
            inputs = layer.activate_neurons(inputs)
        return self.output_layer.activate_neurons(inputs)
    
    def error(self, actual: List[float], expected: List[float]) -> List[float]:
        return [actual[i] - expected[i] for i in range(len(actual))]
    
    def back_propagate(self, inputs: List[float], errors: List[float]) -> None:
        # delta is the derivative of the error functions times the derivate of the activation function
        for index, neuron in enumerate(self.output_layer.neurons):
            neuron.set_delta(errors[index])
        
        for layer_idx, layer in enumerate(reversed(range(len(self.hidden_layers)))):
            next_layer = (
                self.output_layer
                if layer_idx == len(self.hidden_layers) - 1
                else self.hidden_layers[layer_idx + 1]
            )
            for neuron_idx, neuron in enumerate(layer.neurons):
                error_from_next_layer = next_layer.total_delta(neuron_idx)
                neuron.set_delta(error_from_next_layer)
        
        # only update after you've calculated all deltas for all neurons
        self.update_weights_for_all_layers(inputs)

    def update_weights_for_all_layers(self, inputs: List[float]):
        """
        Update weights for all layers
        """
        # Update weights for hidden layers
        for layer_idx in range(len(self.hidden_layers)):
            layer = self.hidden_layers[layer_idx]
            previous_layer_outputs: List[float] = (
                inputs
                if layer_idx == 0
                else self.hidden_layers[layer_idx - 1].all_outputs
            )
            for neuron in layer.neurons:
                self.update_weights_in_a_layer(previous_layer_outputs, neuron)

        # Update weights for output layer
        for index, neuron in enumerate(self.output_layer.neurons):
            self.update_weights_in_a_layer(self.hidden_layers[-1].all_outputs, neuron)

    def update_weights_in_a_layer(
        self, previous_layer_outputs: List[float], neuron: Neuron
    ) -> None:
        """
        Update weights in all neurons in a layer
        """
        for idx in range(len(previous_layer_outputs)):
            neuron.weights[idx] -= (
                self.learning_rate * neuron.delta * previous_layer_outputs[idx]
            )
            neuron.bias -= self.learning_rate * neuron.delta

    def train(
        self,
        num_epoch: int,
        num_outputs: int,
        training_set: List[List[float]],
        training_output: List[float],
    ) -> None:
        for epoch in range(num_epoch):
            sum_error = 0.0
            for idx, row in enumerate(training_set):
                expected = [0 for _ in range(num_outputs)]
                expected[training_output[idx]] = 1  # one-hot encoding
                actual = self.feed_forward(row)
                errors = self.derivative_error_to_output(actual, expected)
                self.back_propagate(row, errors)
                sum_error += self.mse(actual, training_output)
            print(f"Mean squared error: {sum_error}")
            print(f"epoch={epoch}")

    def predict(self, inputs: List[float]) -> int:
        outputs = self.feed_forward(inputs)
        return outputs.index(max(outputs))

    def mse(self, actual: List[float], expected: List[float]) -> float:
        """
        Mean Squared Error formula
        """
        return sum((actual[i] - expected[i]) ** 2 for i in range(len(actual))) / len(
            actual
        )
    

In [11]:
import random

def test_make_prediction_with_network():
    # Test making predictions with the network
    # Mock data is from https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
    dataset = [
        [2.7810836, 2.550537003],
        [1.465489372, 2.362125076],
        [3.396561688, 4.400293529],
        [1.38807019, 1.850220317],
        [3.06407232, 3.005305973],
        [7.627531214, 2.759262235],
        [5.332441248, 2.088626775],
        [6.922596716, 1.77106367],
        [8.675418651, -0.242068655],
        [7.673756466, 3.508563011],
    ]
    expected = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
    n_inputs = len(dataset[0])
    n_outputs = len(set(expected))
    hidden_layers = [
        Layer(
            neurons=[
                Neuron(weights=[random() for _ in range(n_inputs)], bias=random()),
                Neuron(weights=[random() for _ in range(n_inputs)], bias=random()),
            ],
        )
    ]
    output_layer = Layer(
        neurons=[
            Neuron(weights=[random() for _ in range(n_outputs)], bias=random()),
            Neuron(weights=[random() for _ in range(n_outputs)], bias=random()),
        ],
    )
    network = Network(
        hidden_layers=hidden_layers, output_layer=output_layer, learning_rate=0.5
    )
    network.train(40, n_outputs, dataset, expected)
    print(f"Hidden layer: {network.layers[0].neurons}")
    print(f"Output layer: {network.layers[1].neurons}")
    
    # This is just for demonstration only
    for i in range(len(dataset)):
        prediction = network.predict(dataset[i])
        print("Expected=%d, Got=%d" % (expected[i], prediction))

        
if __name__ == "__main__":
    test_make_prediction_with_network()


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?